# Dependency Parsing with Eisner
Par Louis GERARD & Tiago CORTINHAL

In [17]:
import sys
venv_root = '/var/lib/vz/data/g13005245/DEEP/'
sys.path.append(venv_root + 'lib/python3.5/site-packages')

import warnings
warnings.filterwarnings('ignore')

import time
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

## Features 1

We start building a model on these features :
- Governor POS
- Dependant POS
- Distance between these words

### Read data

We read the conllu file to get aall the info we need. In order to make this reusable, we can add any feature to the final list by adding its index to `features_enabled`. Here we have :
- the index, to identify our token
- the part of speech, which we want to pass to our model
- the governor index, to build pairs

In [81]:
def read_conllu(conllu_filename):
    sentences = []

    features = ['INDEX', 'FORM', 'LEMMA', 'POS', 'X1', 'MORPHO', 'GOV', 'LABEL', 'X2', 'X3']
    features_enabled = [0, 3, 6, 7]
    root = [0, 'ROOT', -1, 'root']

    columns = []
    for i, f in enumerate(features):
        if i in features_enabled:
            columns.append(f)

    with open(conllu_filename, 'r', encoding='utf-8') as conllu_file:
        sentence = []
        for i in range(len(features_enabled)):
            sentence.append([root[i]])
        tokens = []
        for line in conllu_file:
            if line[0] == '\n':
                if len(sentence) > 1:
                    sentences.append(sentence)
                    sentence = []
                    for i in range(len(features_enabled)):
                        sentence.append([root[i]])
            elif line[0] == '#':
                pass
            else:
                tokens = line.split('\t')
                if '-' not in tokens[0]:
                    for i, j in enumerate(features_enabled):
                        sentence[i].append(tokens[j])
    return sentences

sentences = read_conllu("UD_French-GSD/fr_gsd-ud-train.conllu")
pd.DataFrame(sentences[3]).T

,0,1,2,3
0,0,ROOT,-1,root
1,1,ADV,5,advmod
2,2,PUNCT,5,punct
3,3,DET,4,det
4,4,NOUN,5,nsubj
5,5,VERB,0,root
6,6,DET,7,det
7,7,NOUN,5,obj
8,8,PUNCT,5,punct


Now we have all the data we need, we can process it to build our dataset. 

We can't pass directly the part of speech tags to our model, so we need to encod it to an one hot vector. We can compute the distances with the indexes. 

We can still reuse this code by adapting `index_i`, `pos_i` and `governor_i` to the previous final result and process the added data in `create_example()`. 

The targets are a bit simplitic because all we need to compute it is the distance. A single neuron with linear activation function, a weight of 1 for the distance and weights of 0 for the POS can do the job. 

In [82]:
def get_sentence(sentences):
    for sentence in sentences:
        yield np.array(sentence).T
        
pos_2_1hot = {
    'ADJ': 1,
    'ADP': 2,
    'ADV': 3,
    'AUX': 4,
    'CCONJ': 5,
    'DET': 6,
    'INTJ': 7,
    'NOUN': 8,
    'NUM': 9,
    'PART': 10,
    'PRON': 11,
    'PROPN': 12,
    'PUNCT': 13,
    'SCONJ': 14,
    'SYM': 15,
    'VERB': 16,
    'X': 17,
    'ROOT': 0
}

labels_2_1hot = {
    'acl': 0,
    'advcl': 1,
    'advmod': 2,
    'amod': 3,
    'appos': 4,
    'aux': 5,
    'case': 6,
    'cc': 7,
    'ccomp': 8,
    'clf': 9,
    'compound': 10,
    'conj': 11,
    'cop': 12,
    'csubj': 13,
    'dep': 14,
    'det': 15,
    'discourse': 16,
    'dislocated': 17,
    'expl': 18,
    'fixed': 19,
    'flat': 20,
    'goeswith': 21,
    'iobj': 22,
    'list': 23,
    'mark': 24,
    'nmod': 25,
    'nsubj': 26,
    'nummod': 27,
    'obj': 28,
    'obl': 29,
    'orphan': 30,
    'parataxis': 31,
    'punct': 32,
    'reparandum': 33,
    'root': 34,
    'vocative': 35,
    'xcomp': 36
}
onehot_2_label = {v: k for k, v in labels_2_1hot.items()}

index_i = 0
pos_i = 1
governor_i = 2
label_i = 3

def create_example(w1, w2, positive=True):
    dist = int(w2[index_i]) - int(w1[index_i])

    pos1 = np.zeros(len(pos_2_1hot))
    pos1[pos_2_1hot[w1[pos_i]]] = 1

    pos2 = np.zeros(len(pos_2_1hot))
    pos2[pos_2_1hot[w2[pos_i]]] = 1
    
    x = np.concatenate(([dist], pos1, pos2))
    label = np.zeros(37)
    
    if positive:
        l = w1[label_i].split(':', 1)[0]
        label[labels_2_1hot[l]] = 1
        y = [dist > 0, dist <= 0]
    else:
        y = [0, 0]
    return x, np.concatenate((y, label))

def create_dataset(sentences, with_negatives=False):
    x = []
    y = []
    for s in get_sentence(sentences):
        for w1 in s:
            if w1[index_i] == '0':
                continue
            w2 = s[int(w1[governor_i])]

            x_token, y_token = create_example(w1, w2)

            x.append(x_token)
            y.append(y_token)
            
            if with_negatives:
                i = np.random.randint(len(s) - 2)
                if i >= int(w1[index_i]):
                    i += 1
                if i >= int(w2[index_i]):
                    i += 1
                w2_negative = s[i]
                x_token, y_token = create_example(w1, w2, False)
                x.append(x_token)
                y.append(y_token)

    return np.array(x), np.array(y)

In [83]:
x, y = create_dataset(sentences)
x.shape, y.shape

((356638, 37), (356638, 39))

### A first model

In [84]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

model = keras.models.Sequential()
model.add(keras.layers.Dense(39, activation='sigmoid', input_dim=37))

tensorboard = keras.callbacks.TensorBoard(log_dir="logs/{}".format(time.time()))

model.compile('sgd', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 39)                1482      
Total params: 1,482
Trainable params: 1,482
Non-trainable params: 0
_________________________________________________________________


In [85]:
h = model.fit(x_train, y_train, epochs=1000, batch_size=1024, validation_data=(x_test, y_test), verbose=0, callbacks=[tensorboard])

x_plt = np.arange(len(h.history['loss']))
plt.plot(x_plt, h.history['loss'], label='train')
plt.plot(x_plt, h.history['val_loss'], label='test')
plt.show()

plt.plot(x_plt, h.history['acc'], label='train')
plt.plot(x_plt, h.history['val_acc'], label='test')
plt.show()

KeyboardInterrupt: 

## Some tests

In [63]:
dist = 1
pos1 = np.zeros(18)
pos1[pos_2_1hot['DET']] = 1
pos2 = np.zeros(18)
pos2[pos_2_1hot['VERB']] = 1
prediction = model.predict(np.concatenate(([dist], pos1, pos2)).reshape(1, -1))[0]
i = np.argmax(prediction[2:])
label = onehot_2_label[i]
label, prediction[i+2], prediction[:2]

('det', 0.50084305, array([0.9817401 , 0.01846319], dtype=float32))

In [64]:
dist = -1
pos1 = np.zeros(18)
pos1[pos_2_1hot['DET']] = 1
pos2 = np.zeros(18)
pos2[pos_2_1hot['VERB']] = 1
prediction = model.predict(np.concatenate(([dist], pos1, pos2)).reshape(1, -1))[0]
i = np.argmax(prediction[2:])
label = onehot_2_label[i]
label, prediction[i+2], prediction[:2]

('det', 0.44818, array([0.09238365, 0.90732414], dtype=float32))

In [65]:
dist = 1
pos1 = np.zeros(18)
pos1[pos_2_1hot['ADJ']] = 1
pos2 = np.zeros(18)
pos2[pos_2_1hot['NOUN']] = 1
prediction = model.predict(np.concatenate(([dist], pos1, pos2)).reshape(1, -1))[0]
i = np.argmax(prediction[2:])
label = onehot_2_label[i]
label, prediction[i+2], prediction[:2]

('amod', 0.4466233, array([0.97494376, 0.0327546 ], dtype=float32))

In [66]:
dist = -1
pos1 = np.zeros(18)
pos1[pos_2_1hot['ADJ']] = 1
pos2 = np.zeros(18)
pos2[pos_2_1hot['NOUN']] = 1
prediction = model.predict(np.concatenate(([dist], pos1, pos2)).reshape(1, -1))[0]
i = np.argmax(prediction[2:])
label = onehot_2_label[i]
label, prediction[i+2], prediction[:2]

('amod', 0.43103153, array([0.06861053, 0.946309  ], dtype=float32))

# Eisner

Now we have our model, we can use it to create the dependency parsing itself with Eisner's algorithm. 

[Explanatory video](https://youtu.be/du9VQaFEyeA)

In [98]:
sentence = np.array(sentences[3]).T
pd.DataFrame(sentence)

,0,1,2,3
0,0,ROOT,-1,root
1,1,ADV,5,advmod
2,2,PUNCT,5,punct
3,3,DET,4,det
4,4,NOUN,5,nsubj
5,5,VERB,0,root
6,6,DET,7,det
7,7,NOUN,5,obj
8,8,PUNCT,5,punct


In [122]:
def eisner(sentence, model):
    n = sentence.shape[0]

    full_left = []
    full_right = []
    full_max = []
    part_left = []
    part_right = []
    part_left_max = []
    part_right_max = []
    right_label = []
    left_label = []
    for i in range(n):
        full_left.append([0])
        full_right.append([0])
        full_max.append([0])
        part_left.append([0])
        part_right.append([0])
        part_left_max.append([0])
        part_right_max.append([0])
        right_label.append([])
        left_label.append([])

    for m in range(1, n):
        for i1 in range(n - m):
            i2 = i1 + m

            x, _ = create_example(sentence[i1], sentence[i2])
            score = model.predict(x.reshape(1, -1))[0]
            
            score[0] = 0
            score[1] = 0
            if sentence[i1][governor_i] == sentence[i2][index_i]:
                score[0] = 1
            elif sentence[i2][governor_i] == sentence[i1][index_i]:
                score[1] = 1
                
            i = np.argmax(prediction[2:])
            label = onehot_2_label[i]

            max_full = -1
            max_q = -1
            for q in range(i2 - i1):
                q_line = q + i1
                current = full_left[i1][q] + full_right[q_line + 1][i2 - q_line - 1]
                if current > max_full:
                    max_full = current
                    max_q = q_line
            
            part_left[i1].append(max_full + score[1])
            part_right[i1].append(max_full + score[0])
            full_max[i1].append(max_q)
            
            max_part_l = -1
            max_part_r = -1
            max_q_l = -1
            max_q_r = -1
            for q in range(i2 - i1):
                q_line = q + i1
                current_l = full_left[i1][q] + part_left[q_line][i2 - q_line]
                current_r = part_right[i1][q + 1] + full_right[q_line + 1][i2 - q_line - 1]

                if current_l > max_part_l:
                    max_part_l = current_l
                    max_q_l = q_line
                if current_r > max_part_r:
                    max_part_r = current_r
                    max_q_r = q_line + 1

            full_left[i1].append(max_part_l)
            full_right[i1].append(max_part_r)
            part_left_max[i1].append(max_q_l)
            part_right_max[i1].append(max_q_r)
            
    return full_left, full_right, full_max, part_left, part_right, part_left_max, part_right_max

full_left, full_right, full_max, part_left, part_right, part_left_max, part_right_max = eisner(sentence, model)

In [123]:
pd.DataFrame(full_left)

,0,1,2,3,4,5,6,7,8
0,0,0.0,0.0,0.0,1.0,5.0,5.0,7.0,8.0
1,0,0.0,0.0,1.0,3.0,3.0,5.0,6.0,NaN
2,0,0.0,1.0,2.0,2.0,4.0,5.0,NaN,NaN
3,0,0.0,1.0,1.0,3.0,4.0,NaN,NaN,NaN
4,0,0.0,0.0,2.0,3.0,NaN,NaN,NaN,NaN
5,0,0.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN
6,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
pd.DataFrame(full_right)

,0,1,2,3,4,5,6,7,8
0,0,0.0,0.0,0.0,1.0,4.0,5.0,6.0,7.0
1,0,0.0,0.0,1.0,4.0,4.0,5.0,6.0,NaN
2,0,0.0,1.0,3.0,3.0,4.0,5.0,NaN,NaN
3,0,1.0,2.0,2.0,3.0,4.0,NaN,NaN,NaN
4,0,1.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN
5,0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
6,0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
pd.DataFrame(part_left)

,0,1,2,3,4,5,6,7,8
0,0,0.0,0.0,0.0,1.0,5.0,5.0,6.0,7.0
1,0,0.0,0.0,1.0,3.0,3.0,4.0,5.0,NaN
2,0,0.0,1.0,2.0,2.0,3.0,4.0,NaN,NaN
3,0,0.0,1.0,1.0,2.0,3.0,NaN,NaN,NaN
4,0,0.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN
5,0,0.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN
6,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
pd.DataFrame(part_right)

,0,1,2,3,4,5,6,7,8
0,0,0.0,0.0,0.0,1.0,4.0,5.0,6.0,7.0
1,0,0.0,0.0,1.0,4.0,3.0,4.0,5.0,NaN
2,0,0.0,1.0,3.0,2.0,3.0,4.0,NaN,NaN
3,0,1.0,1.0,1.0,2.0,3.0,NaN,NaN,NaN
4,0,1.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN
5,0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
6,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
pd.DataFrame(full_max)

,0,1,2,3,4,5,6,7,8
0,0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,7.0
1,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2,0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN
3,0,3.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN
4,0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN
5,0,5.0,5.0,7.0,NaN,NaN,NaN,NaN,NaN
6,0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
pd.DataFrame(part_left_max)

,0,1,2,3,4,5,6,7,8
0,0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0
1,0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,NaN
2,0,2.0,2.0,2.0,2.0,5.0,5.0,NaN,NaN
3,0,3.0,3.0,3.0,5.0,5.0,NaN,NaN,NaN
4,0,4.0,4.0,5.0,5.0,NaN,NaN,NaN,NaN
5,0,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN
6,0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
